In [ ]:
# Load dependent libraries
import warnings
warnings.filterwarnings('ignore')
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
import sys
sys.path.append('..')
import sys
from A_1k_arg_sampling import ReverseDiffusionPredictor,AnnealedLangevinDynamics
import aapm_sin_ncsnpp_gb as configs
sys.path.append('..')
from losses import get_optimizer
from models.ema import ExponentialMovingAverage
from utils import restore_checkpoint
from sde_lib import VESDE
import os
import sys
sys.path.append('..')
from models import utils as mutils

In [2]:
# Set the model parameters and load the network
ckpt_filename ='./exp/checkpoints/checkpoint_23.pth'
assert os.path.exists(ckpt_filename)
config = configs.get_config()  
sde = VESDE(sigma_min=config.model.sigma_min, sigma_max=config.model.sigma_max, N=config.model.num_scales)
sampling_eps = 1e-5
batch_size = 1 
config.training.batch_size = batch_size
config.eval.batch_size = batch_size

random_seed = 0 

sigmas = mutils.get_sigmas(config)
score_model = mutils.create_model(config)

optimizer = get_optimizer(config, score_model.parameters())
ema = ExponentialMovingAverage(score_model.parameters(), decay=config.model.ema_rate)
state = dict(step=0, optimizer=optimizer, model=score_model, ema=ema)

state = restore_checkpoint(ckpt_filename, state, config.device)
ema.copy_to(score_model.parameters())

img_size = config.data.image_size
channels = config.data.num_channels
shape = (batch_size, channels, img_size, img_size)
snr = 0.02734
n_steps =  2
probability_flow = False


In [ ]:
# Image reconstruction is under way
import sampling
sampling_fn = sampling.get_pc_sampler(sde, ReverseDiffusionPredictor, AnnealedLangevinDynamics,
                                      None, snr, n_steps=n_steps,
                                      probability_flow=probability_flow,
                                      continuous=config.training.continuous,
                                      eps=sampling_eps, device=config.device)

sampling_fn(score_model,32,ReverseDiffusionPredictor,AnnealedLangevinDynamics)


size 500 gap 60 Net True sparse False amp 0  psnr 14.9700  ssim 0.6122  mse 0.0318
size 500 gap 60 Net True sparse False phase 0  psnr 14.1700 ssim 0.3567  mse 0.0383
size 500 gap 60 Net True sparse False amp 5  psnr 16.1300  ssim 0.6715  mse 0.0244
size 500 gap 60 Net True sparse False phase 5  psnr 14.5800 ssim 0.3674  mse 0.0349
size 500 gap 60 Net True sparse False amp 10  psnr 16.7300  ssim 0.6963  mse 0.0213
size 500 gap 60 Net True sparse False phase 10  psnr 14.9000 ssim 0.3873  mse 0.0324
